<a href="https://colab.research.google.com/github/ancestor9/24_fall_python-programimming-2/blob/main/1119_03_10_Working_With_Strings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vectorized String Operations

One strength of Python is its relative ease in handling and manipulating string data.
Pandas builds on this and provides a comprehensive set of *vectorized string operations* that are an important part of the type of munging required when working with (read: cleaning up) real-world data.
In this chapter, we'll walk through some of the Pandas string operations, and then take a look at using them to partially clean up a very messy dataset of recipes collected from the internet.

## Introducing Pandas String Operations

We saw in previous chapters how tools like NumPy and Pandas generalize arithmetic operations so that we can easily and quickly perform the same operation on many array elements. For example:

In [ ]:
import numpy as np
x = np.array([2, 3, 5, 7, 11, 13])
x * 2

array([ 4,  6, 10, 14, 22, 26])

This *vectorization* of operations simplifies the syntax of operating on arrays of data: we no longer have to worry about the size or shape of the array, but just about what operation we want done.
For arrays of strings, NumPy does not provide such simple access, and thus you're stuck using a more verbose loop syntax:

In [ ]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']

['Peter', 'Paul', 'Mary', 'Guido']

This is perhaps sufficient to work with some data, but it will break if there are any missing values, so this approach requires putting in extra checks:

In [ ]:
data = ['peter', 'Paul', None, 'MARY', 'gUIDO']

['Peter', 'Paul', None, 'Mary', 'Guido']

This kind of manual approach is not only verbose and inconvenient, it can be error-prone.

Pandas includes features to address both this need for vectorized string operations and the need for correctly handling missing data via the `str` attribute of Pandas `Series` and `Index` objects containing strings.
So, for example, if we create a Pandas `Series` with this data we can directly call the `str.capitalize` method, which has missing value handling built in:

In [ ]:
import pandas as pd
names = pd.Series(data)
names.str.capitalize()

,0
0,Peter
1,Paul
2,None
3,Mary
4,Guido


## Tables of Pandas String Methods

If you have a good understanding of string manipulation in Python, most of the Pandas string syntax is intuitive enough that it's probably sufficient to just list the available methods. We'll start with that here, before diving deeper into a few of the subtleties.
The examples in this section use the following `Series` object:

In [ ]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam',
                   'Eric Idle', 'Terry Jones', 'Michael Palin'])

### Methods Similar to Python String Methods

Nearly all of Python's built-in string methods are mirrored by a Pandas vectorized string method. Here is a list of Pandas `str` methods that mirror Python string methods:

|           |                |                |                |
|-----------|----------------|----------------|----------------|
|`len()`    | `lower()`      | `translate()`  | `islower()`    |
|`ljust()`  | `upper()`      | `startswith()` | `isupper()`    |
|`rjust()`  | `find()`       | `endswith()`   | `isnumeric()`  |
|`center()` | `rfind()`      | `isalnum()`    | `isdecimal()`  |
|`zfill()`  | `index()`      | `isalpha()`    | `split()`      |
|`strip()`  | `rindex()`     | `isdigit()`    | `rsplit()`     |
|`rstrip()` | `capitalize()` | `isspace()`    | `partition()`  |
|`lstrip()` | `swapcase()`   | `istitle()`    | `rpartition()` |

Notice that these have various return values. Some, like `lower`, return a series of strings:

,0
0,graham chapman
1,john cleese
2,terry gilliam
3,eric idle
4,terry jones
5,michael palin


But some others return numbers:

,0
0,14
1,11
2,13
3,9
4,11
5,13


Or Boolean values:

,0
0,False
1,False
2,True
3,False
4,True
5,False


Still others return lists or other compound values for each element:

,0
0,"[Graham, Chapman]"
1,"[John, Cleese]"
2,"[Terry, Gilliam]"
3,"[Eric, Idle]"
4,"[Terry, Jones]"
5,"[Michael, Palin]"


## **Task**

In [ ]:
df = pd.DataFrame(monte.str.split())
df

,0
0,"[Graham, Chapman]"
1,"[John, Cleese]"
2,"[Terry, Gilliam]"
3,"[Eric, Idle]"
4,"[Terry, Jones]"
5,"[Michael, Palin]"


,0
0,graham chapman
1,john cleese
2,terry gilliam
3,eric idle
4,terry jones
5,michael palin


We'll see further manipulations of this kind of series-of-lists object as we continue our discussion.

### Methods Using Regular Expressions

In addition, there are several methods that accept regular expressions (regexps) to examine the content of each string element, and follow some of the API conventions of Python's built-in `re` module:

| Method    | Description |
|-----------|-------------|
| `match`   | Calls `re.match` on each element, returning a Boolean. |
| `extract` | Calls `re.match` on each element, returning matched groups as strings.|
| `findall` | Calls `re.findall` on each element |
| `replace` | Replaces occurrences of pattern with some other string|
| `contains`| Calls `re.search` on each element, returning a boolean |
| `count`   | Counts occurrences of pattern|
| `split`   | Equivalent to `str.split`, but accepts regexps |
| `rsplit`  | Equivalent to `str.rsplit`, but accepts regexps |

With these, we can do a wide range of operations.
For example, we can extract the first name from each element by asking for a contiguous group of characters at the beginning of each element:

In [ ]:
monte

,0
0,Graham Chapman
1,John Cleese
2,Terry Gilliam
3,Eric Idle
4,Terry Jones
5,Michael Palin


In [ ]:
monte.str.extract('([A-Za-z]+)')

,0
0,Graham
1,John
2,Terry
3,Eric
4,Terry
5,Michael


In [ ]:
monte.str.extract('([A-Za-z]+)', expand=True)

,0
0,Graham
1,John
2,Terry
3,Eric
4,Terry
5,Michael


Or we can do something more complicated, like finding all names that start and end with a consonant, making use of the start-of-string (`^`) and end-of-string (`$`) regular expression characters:

In [ ]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

,0
0,[Graham Chapman]
1,[]
2,[Terry Gilliam]
3,[]
4,[Terry Jones]
5,[Michael Palin]


The ability to concisely apply regular expressions across `Series` or `DataFrame` entries opens up many possibilities for analysis and cleaning of data.

### Miscellaneous Methods
Finally, there are some miscellaneous methods that enable other convenient operations:

| Method | Description |
|--------|-------------|
| `get` | Indexes each element |
| `slice` | Slices each element|
| `slice_replace` | Replaces slice in each element with the passed value|
| `cat`      | Concatenates strings|
| `repeat` | Repeats values |
| `normalize` | Returns Unicode form of strings |
| `pad` | Adds whitespace to left, right, or both sides of strings|
| `wrap` | Splits long strings into lines with length less than a given width|
| `join` | Joins strings in each element of the `Series` with the passed separator|
| `get_dummies` | Extracts dummy variables as a `DataFrame` |

#### Vectorized item access and slicing

The `get` and `slice` operations, in particular, enable vectorized element access from each array.
For example, we can get a slice of the first three characters of each array using `str.slice(0, 3)`.
Note that this behavior is also available through Python's normal indexing syntax; for example, `df.str.slice(0, 3)` is equivalent to `df.str[0:3]`:

In [ ]:
monte.str[0:3]

,0
0,Gra
1,Joh
2,Ter
3,Eri
4,Ter
5,Mic


Indexing via `df.str.get(i)` and `df.str[i]` are likewise similar.

These indexing methods also let you access elements of arrays returned by `split`.
For example, to extract the last name of each entry, we can combine `split` with `str` indexing:

,0
0,Chapman
1,Cleese
2,Gilliam
3,Idle
4,Jones
5,Palin


#### Indicator variables

Another method that requires a bit of extra explanation is the `get_dummies` method.
This is useful when your data has a column containing some sort of coded indicator.
For example, we might have a dataset that contains information in the form of codes, such as A = "born in America," B = "born in the United Kingdom," C = "likes cheese," D = "likes spam":

In [ ]:
full_monte = pd.DataFrame({'name': monte,
                           'info': ['B|C|D', 'B|D', 'A|C',
                                    'B|D', 'B|C', 'B|C|D']})
full_monte

,name,info
0,Graham Chapman,B|C|D
1,John Cleese,B|D
2,Terry Gilliam,A|C
3,Eric Idle,B|D
4,Terry Jones,B|C
5,Michael Palin,B|C|D


The `get_dummies` routine lets us split out these indicator variables into a `DataFrame`:

In [ ]:
full_monte['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


## **따릉이 데이터**
- **[데이터 받기](https://data.seoul.go.kr/dataList/OA-15182/F/1/datasetView.do)**

In [ ]:
path = '/content/서울특별시 공공자전거 대여이력 정보_2406.csv'

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),생년,성별,이용자종류,대여대여소ID,반납대여소ID,자전거구분
0,SPB-32912,2024-06-01 00:00:04,602,장안동 사거리,0,2024-06-01 00:00:57,00602,장안동 사거리,0,0,0.00,1968,F,내국인,ST-262,ST-262,일반자전거
1,SPB-40904,2024-06-01 00:01:34,3504,원일교회,0,2024-06-01 00:01:49,03504,원일교회,0,0,141.91,1992,F,내국인,ST-987,ST-987,일반자전거
2,SPB-54570,2024-06-01 00:01:40,4836,양원역 1번 출구,0,2024-06-01 00:02:13,04836,양원역 1번 출구,0,0,0.00,1987,M,내국인,ST-3023,ST-3023,일반자전거
3,SPB-64295,2024-06-01 00:00:04,1163,방화동강서기동대앞,0,2024-06-01 00:02:38,03772,길성그랑프리텔아파트,0,2,0.00,2007,M,내국인,ST-1254,ST-2482,일반자전거
4,SPB-58205,2024-06-01 00:00:25,268,그랜드컨벤션센터 앞,0,2024-06-01 00:02:45,00228,선유도역 3번출구 앞,0,2,0.00,1970,\N,내국인,ST-422,ST-278,일반자전거


In [ ]:
df.columns

Index(['자전거번호', '대여일시', '대여 대여소번호', '대여 대여소명', '대여거치대', '반납일시', '반납대여소번호',
       '반납대여소명', '반납거치대', '이용시간(분)', '이용거리(M)', '생년', '성별', '이용자종류', '대여대여소ID',
       '반납대여소ID', '자전거구분'],
      dtype='object')

In [ ]:
df = df[['대여 대여소명', '반납대여소명', '이용시간(분)', '이용거리(M)', '생년', '성별', '이용자종류', ]]
df

,대여 대여소명,반납대여소명,이용시간(분),이용거리(M),생년,성별,이용자종류
0,장안동 사거리,장안동 사거리,0,0.00,1968,F,내국인
1,원일교회,원일교회,0,141.91,1992,F,내국인
2,양원역 1번 출구,양원역 1번 출구,0,0.00,1987,M,내국인
3,방화동강서기동대앞,길성그랑프리텔아파트,2,0.00,2007,M,내국인
4,그랜드컨벤션센터 앞,선유도역 3번출구 앞,2,0.00,1970,\N,내국인
...,...,...,...,...,...,...,...
5004325,종로3가역 2번출구 뒤,\N,60,4348.00,\N,\N,외국인
5004326,한강공원 망원나들목,\N,60,32253.00,\N,\N,외국인
5004327,건대병원후문,\N,870,848.00,\N,\N,비회원
5004328,반포래미안 리오센트,\N,870,2119.00,1998,F,내국인


,대여 대여소명,반납대여소명,이용시간(분),이용거리(M),생년,성별,이용자종류
25,자양(뚝섬한강공원)역 1번출구 앞,자양나들목,4,743.14,2004,F,내국인
48,동대문역사문화공원역 1번출구 뒤편,동대문역사문화공원역 1번출구 뒤편,1,0.00,\N,\N,외국인
114,길음8골어린이공원 옆,성북제일새마을금고 본점 앞,2,0.00,1960,M,내국인
158,은평평화공원(역촌역4번출구),녹번 힐스테이트상가앞,9,1526.10,1978,M,내국인
174,자양(뚝섬한강공원)역 1번출구 앞,자양한강도서관,5,998.56,1999,\N,내국인
...,...,...,...,...,...,...,...
5004218,풍납백제문화공원 옆 인근,포레스 주상복합 빌딩,77,1332.02,1988,M,내국인
5004233,한강공원 망원나들목,삼성전자서비스 성북센터,86,23120.00,1974,\N,내국인
5004290,구의공원(테크노마트 앞),성내어울터,211,4980.91,1988,\N,내국인
5004308,한강공원 망원나들목,옥수역 3번출구,743,27762.77,1996,M,내국인


,대여 대여소명,반납대여소명,이용시간(분),이용거리(M),생년,성별,이용자종류,공간
5004320,잠실나들목,\N,60,7471.0,\N,\N,외국인,NaN
5004321,한성백제역 2번 출구,\N,60,35073.0,\N,\N,외국인,NaN
5004322,역촌파출소,\N,60,1345.0,1962,M,내국인,NaN
5004323,노원역1번출구,노원정보도서관,45,0.0,\N,\N,외국인,NaN
5004324,서강대 정문 건너편,\N,60,3633.0,\N,\N,외국인,NaN
5004325,종로3가역 2번출구 뒤,\N,60,4348.0,\N,\N,외국인,NaN
5004326,한강공원 망원나들목,\N,60,32253.0,\N,\N,외국인,공원
5004327,건대병원후문,\N,870,848.0,\N,\N,비회원,NaN
5004328,반포래미안 리오센트,\N,870,2119.0,1998,F,내국인,NaN
5004329,양재시민의숲역 1번출구,\N,870,0.0,1988,M,내국인,NaN


In [ ]:
for text in ['운동장', '빌딩', '정류장']:
    df.loc[df['대여 대여소명'].str.contains(text), '공간'] = text

In [ ]:
df.sample(10)

,대여 대여소명,반납대여소명,이용시간(분),이용거리(M),생년,성별,이용자종류,공간
3741069,노들역 3번출구,흑석역 1번출구,71,1504.10,1997,M,내국인,역
2024485,구로디지털단지역 앞,태평양물산,8,713.50,1992,F,내국인,역
4648885,마곡나루역 3번 출구,대상연구소,2,222.39,1997,F,내국인,역
2581010,대림아크로리버 앞,대림아크로리버 앞,72,13030.00,1992,\N,내국인,NaN
34660,도림고가도로 밑,영등포구청역 1번출구,25,2319.37,1984,F,내국인,NaN
3371976,가산동 주민센터,에이스 하드웨어(시흥대로 396) 앞,4,1092.31,1999,M,내국인,NaN
413954,둔촌오륜역 1번 출구,둔촌 주공 GS 맞은편,2,0.00,\N,\N,내국인,역
749411,한강아파트,강서한강자이타워 앞,6,900.00,2007,F,내국인,NaN
2085494,우이교 교차로,도봉한신아파트 버스정류장,29,4795.56,1984,M,내국인,NaN
1866291,화곡역 1번 출구 앞,강서구립등빛도서관,42,3829.75,2003,M,내국인,역


<ipython-input-58-9d81f069809c>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['공간'].fillna('기타', inplace=True)


,대여 대여소명,반납대여소명,이용시간(분),이용거리(M),생년,성별,이용자종류,공간
0,장안동 사거리,장안동 사거리,0,0.00,1968,F,내국인,기타
1,원일교회,원일교회,0,141.91,1992,F,내국인,기타
2,양원역 1번 출구,양원역 1번 출구,0,0.00,1987,M,내국인,역
3,방화동강서기동대앞,길성그랑프리텔아파트,2,0.00,2007,M,내국인,기타
4,그랜드컨벤션센터 앞,선유도역 3번출구 앞,2,0.00,1970,\N,내국인,기타
...,...,...,...,...,...,...,...,...
5004325,종로3가역 2번출구 뒤,\N,60,4348.00,\N,\N,외국인,역
5004326,한강공원 망원나들목,\N,60,32253.00,\N,\N,외국인,공원
5004327,건대병원후문,\N,870,848.00,\N,\N,비회원,기타
5004328,반포래미안 리오센트,\N,870,2119.00,1998,F,내국인,기타


,생년
공간,
공원,245503
기타,3071272
빌딩,101436
역,1444071
운동장,10361
정류장,131687


,이용시간(분),이용거리(M)
공간,,
공원,23.914579,2861.884664
기타,21.012424,2457.227536
빌딩,20.342699,2276.058263
역,21.233354,2420.336709
운동장,24.562494,2855.822559
정류장,19.748419,2250.920654


In [ ]:
df.shape

(5004330, 8)

In [ ]:
df = df.sample(frac=0.1)

In [ ]:
# prompt: fake를 사용하여 df크기만큼 주소를 생성하여 '주소' 컬럼을 만들어라 . 대한민국 주소로

!pip install faker --quiet

from faker import Faker

fake = Faker('ko_KR') # Use Korean locale

# Generate addresses
addresses = [fake.address() for _ in range(len(df))]
df['주소'] = addresses

df

,대여 대여소명,반납대여소명,이용시간(분),이용거리(M),생년,성별,이용자종류,공간,주소
2021101,양평1 보행육교 앞,이앤씨드림타워 앞,8,1206.00,\N,F,내국인,기타,경상북도 홍성군 강남대97길
3574795,한내 행복발전소 옆,태릉입구역 3번출구,20,2308.76,2004,M,내국인,기타,경상북도 부여군 영동대거리
3449391,항동지구 6단지 602동 앞,항동지구 11단지 1103동 앞,45,279.83,2002,\N,내국인,기타,경상남도 괴산군 백제고분길
1931462,보문4교 인근,보문역6번출구 앞,2,0.00,1995,F,내국인,기타,경기도 양주시 잠실86거리
1202930,유니베라 앞,성수사거리 버스정류장 앞,7,990.00,1999,F,내국인,기타,강원도 고양시 논현거리 (미경이마을)
...,...,...,...,...,...,...,...,...,...
3896411,공릉역3번출구,수락산역 4번출구,36,7336.37,1967,M,내국인,역,제주특별자치도 청주시 상당구 반포대62길
4525012,샛강역 1번출구 앞,U레지던스 앞,8,910.56,1986,M,내국인,역,전라남도 성남시 수정구 강남대5길
3016675,종로3가역 8번출구,포르투갈 대사관 앞,5,797.45,1991,M,내국인,역,제주특별자치도 청주시 상당구 역삼가
624538,신림역 8번출구,신림역 8번출구,0,0.00,1979,M,내국인,역,대구광역시 성북구 도산대로


In [ ]:
df.duplicated('주소').sum()

63847

In [ ]:
df.drop_duplicates('주소', inplace=True)
df

,대여 대여소명,반납대여소명,이용시간(분),이용거리(M),생년,성별,이용자종류,공간,주소
0,장안동 사거리,장안동 사거리,0,0.00,1968,F,내국인,기타,충청남도 공주시 역삼15거리
1,원일교회,원일교회,0,141.91,1992,F,내국인,기타,경기도 천안시 서북구 도산대로
2,양원역 1번 출구,양원역 1번 출구,0,0.00,1987,M,내국인,역,대구광역시 강남구 개포21거리
3,방화동강서기동대앞,길성그랑프리텔아파트,2,0.00,2007,M,내국인,기타,부산광역시 성북구 테헤란0로
4,그랜드컨벤션센터 앞,선유도역 3번출구 앞,2,0.00,1970,\N,내국인,기타,부산광역시 강동구 역삼가
...,...,...,...,...,...,...,...,...,...
183983,염창동 한마음아파트 앞,안양천합수부(현대3차아파트),15,1784.83,2008,M,내국인,기타,경상남도 가평군 서초대76거리 (영숙김면)
183984,자양(뚝섬한강공원)역 1번출구 앞,웰츠타워 오피스텔,17,2040.46,1994,M,내국인,역,부산광역시 중랑구 백제고분247길 (상철박김동)
183985,청계천 영도교,신당역 12번 출구 뒤,10,851.17,2007,M,내국인,기타,강원도 이천시 양재천길
183986,송정역 1번출구,방화사거리 마을버스 버스정류장,6,1380.00,1997,M,내국인,역,충청남도 춘천시 언주5로


In [ ]:
df['시도'] = df['주소'].???
df['시군구'] = df['주소'].????

In [ ]:
df = pd.get_dummies(df, columns=['성별', '이용자종류', '공간', '시도', '시군구']).drop(columns=['대여 대여소명', '반납대여소명', '주소'])
df*1

,이용시간(분),이용거리(M),생년,성별_F,성별_M,성별_\N,성별_f,성별_m,이용자종류_내국인,이용자종류_비회원,...,시군구_파주시,시군구_평창군,시군구_평택시,시군구_포천시,시군구_하남시,시군구_홍성군,시군구_홍천군,시군구_화성시,시군구_화천군,시군구_횡성군
2021101,8,1206.00,\N,1,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3574795,20,2308.76,2004,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3449391,45,279.83,2002,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1931462,2,0.00,1995,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1202930,7,990.00,1999,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3896411,36,7336.37,1967,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4525012,8,910.56,1986,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3016675,5,797.45,1991,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
624538,0,0.00,1979,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df['생년'].isnull().sum()

0

In [ ]:
df['생년'].unique()

array(['\\N', '2004', '2002', '1995', '1999', '1985', '1972', '1996',
       '1998', '1993', '1982', '1980', '2010', '1973', '1992', '1968',
       '1969', '1977', '1994', '1970', '1986', '2005', '1981', '1989',
       '1984', '1991', '1967', '1997', '1974', '1987', '2001', '2000',
       '1965', '1960', '1966', '1988', '1978', '2006', '1971', '1983',
       '1990', '2078', '2009', '2007', '2003', '1976', '1901', '2008',
       '1963', '1979', '1958', '1962', '1975', '1959', '1954', '1957',
       '1956', '1961', '1941', '1964', '1952', '1955', '1907', '1902',
       '2069', '1951', '1943', '2058', '2095', '1934', '2098', '1953',
       '1940', '1933', '1936', '1950', '1947', '1945', '1949', '1948',
       '0000', '2011', '1937', '2073', '2085', '2019', '2047', '2059',
       '2021', '2016', '1944', '2020', '2094', '2088', '2076', '2017',
       '2097', '2075', '2050', '1942', '1946', '2062', '2068', '2049',
       '1911', '2070', '2018', '1906', '2014', '1939', '2079', '2081',
       

In [ ]:
df[df['생년'] == '\\N']

,이용시간(분),이용거리(M),생년,성별_F,성별_M,성별_\N,성별_f,성별_m,이용자종류_내국인,이용자종류_비회원,...,시군구_파주시,시군구_평창군,시군구_평택시,시군구_포천시,시군구_하남시,시군구_홍성군,시군구_홍천군,시군구_화성시,시군구_화천군,시군구_횡성군
2021101,8,1206.00,\N,True,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False
1332586,16,1563.48,\N,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2580326,4,523.11,\N,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2835191,6,1000.20,\N,True,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False
74750,4,840.00,\N,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244724,14,1572.83,\N,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
309707,49,6776.00,\N,False,False,True,False,False,False,True,...,True,False,False,False,False,False,False,False,False,False
4436213,117,7830.33,\N,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3514720,7,964.80,\N,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
df['생년'].replace('\\N', np.nan, inplace=True)
df.dropna(subset=['생년'], inplace=True)
df

,이용시간(분),이용거리(M),생년,성별_F,성별_M,성별_\N,성별_f,성별_m,이용자종류_내국인,이용자종류_비회원,...,시군구_파주시,시군구_평창군,시군구_평택시,시군구_포천시,시군구_하남시,시군구_홍성군,시군구_홍천군,시군구_화성시,시군구_화천군,시군구_횡성군
3574795,20,2308.76,2004,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3449391,45,279.83,2002,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1931462,2,0.00,1995,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1202930,7,990.00,1999,True,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1014667,9,1318.25,1985,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3896411,36,7336.37,1967,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4525012,8,910.56,1986,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3016675,5,797.45,1991,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
624538,0,0.00,1979,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
import datetime

today = datetime.date.today()
today

datetime.date(2024, 11, 10)

In [ ]:
today.year

2024

In [ ]:
# df['생년'] - today.year
today.year - df['생년'].astype('int')

,생년
3574795,20
3449391,22
1931462,29
1202930,25
1014667,39
...,...
3896411,57
4525012,38
3016675,33
624538,45


In [ ]:
df['연령'] = today.year - df['생년'].astype('int16')
df.drop(columns=['생년'], inplace=True)
df

,이용시간(분),이용거리(M),성별_F,성별_M,성별_\N,성별_f,성별_m,이용자종류_내국인,이용자종류_비회원,이용자종류_외국인,...,시군구_평창군,시군구_평택시,시군구_포천시,시군구_하남시,시군구_홍성군,시군구_홍천군,시군구_화성시,시군구_화천군,시군구_횡성군,연령
3574795,20,2308.76,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,20
3449391,45,279.83,False,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,22
1931462,2,0.00,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,29
1202930,7,990.00,True,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,25
1014667,9,1318.25,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3896411,36,7336.37,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,57
4525012,8,910.56,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,38
3016675,5,797.45,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,33
624538,0,0.00,False,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,45


With these operations as building blocks, you can construct an endless range of string processing procedures when cleaning your data.

We won't dive further into these methods here, but I encourage you to read through ["Working with Text Data"](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html) in the Pandas online documentation, or to refer to the resources listed in [Further Resources](03.13-Further-Resources.ipynb).

## Example: Recipe Database

These vectorized string operations become most useful in the process of cleaning up messy, real-world data.
Here I'll walk through an example of that, using an open recipe database compiled from various sources on the web.
Our goal will be to parse the recipe data into ingredient lists, so we can quickly find a recipe based on some ingredients we have on hand. The scripts used to compile this can be found at https://github.com/fictivekin/openrecipes, and the link to the most recent version of the database is found there as well.

This database is about 30 MB, and can be downloaded and unzipped with these commands:

In [ ]:
! mkdir data

In [ ]:
repo = "https://raw.githubusercontent.com/jakevdp/open-recipe-data/master"
!cd data && curl -O {repo}/recipeitems.json.gz
!gunzip data/recipeitems.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 29.3M  100 29.3M    0     0  18.8M      0  0:00:01  0:00:01 --:--:-- 18.8M


The database is in JSON format, so we will use `pd.read_json` to read it (`lines=True` is required for this dataset because each line of the file is a JSON entry):

In [ ]:
recipes = pd.read_json('data/recipeitems.json', lines=True)
recipes.shape

(173278, 17)

In [ ]:
recipes.head()

,_id,name,ingredients,url,image,ts,cookTime,source,recipeYield,datePublished,prepTime,description,totalTime,creator,recipeCategory,dateModified,recipeInstructions
0,{'$oid': '5160756b96cc62079cc2db15'},Drop Biscuits and Sausage Gravy,Biscuits\n3 cups All-purpose Flour\n2 Tablespo...,http://thepioneerwoman.com/cooking/2013/03/dro...,http://static.thepioneerwoman.com/cooking/file...,{'$date': 1365276011104},PT30M,thepioneerwoman,12,2013-03-11,PT10M,"Late Saturday afternoon, after Marlboro Man ha...",NaN,NaN,NaN,NaN,NaN
1,{'$oid': '5160756d96cc62079cc2db16'},Hot Roast Beef Sandwiches,12 whole Dinner Rolls Or Small Sandwich Buns (...,http://thepioneerwoman.com/cooking/2013/03/hot...,http://static.thepioneerwoman.com/cooking/file...,{'$date': 1365276013902},PT20M,thepioneerwoman,12,2013-03-13,PT20M,"When I was growing up, I participated in my Ep...",NaN,NaN,NaN,NaN,NaN
2,{'$oid': '5160756f96cc6207a37ff777'},Morrocan Carrot and Chickpea Salad,Dressing:\n1 tablespoon cumin seeds\n1/3 cup /...,http://www.101cookbooks.com/archives/moroccan-...,http://www.101cookbooks.com/mt-static/images/f...,{'$date': 1365276015332},NaN,101cookbooks,NaN,2013-01-07,PT15M,A beauty of a carrot salad - tricked out with ...,NaN,NaN,NaN,NaN,NaN
3,{'$oid': '5160757096cc62079cc2db17'},Mixed Berry Shortcake,Biscuits\n3 cups All-purpose Flour\n2 Tablespo...,http://thepioneerwoman.com/cooking/2013/03/mix...,http://static.thepioneerwoman.com/cooking/file...,{'$date': 1365276016700},PT15M,thepioneerwoman,8,2013-03-18,PT15M,It's Monday! It's a brand new week! The birds ...,NaN,NaN,NaN,NaN,NaN
4,{'$oid': '5160757496cc6207a37ff778'},Pomegranate Yogurt Bowl,For each bowl: \na big dollop of Greek yogurt\...,http://www.101cookbooks.com/archives/pomegrana...,http://www.101cookbooks.com/mt-static/images/f...,{'$date': 1365276020318},NaN,101cookbooks,Serves 1.,2013-01-20,PT5M,A simple breakfast bowl made with Greek yogurt...,NaN,NaN,NaN,NaN,NaN


We see there are nearly 175,000 recipes, and 17 columns.
Let's take a look at one row to see what we have:

In [ ]:
recipes.iloc[0]

,0
_id,{'$oid': '5160756b96cc62079cc2db15'}
name,Drop Biscuits and Sausage Gravy
ingredients,Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url,http://thepioneerwoman.com/cooking/2013/03/dro...
image,http://static.thepioneerwoman.com/cooking/file...
ts,{'$date': 1365276011104}
cookTime,PT30M
source,thepioneerwoman
recipeYield,12
datePublished,2013-03-11


There is a lot of information there, but much of it is in a very messy form, as is typical of data scraped from the web.
In particular, the ingredient list is in string format; we're going to have to carefully extract the information we're interested in.
Let's start by taking a closer look at the ingredients:

In [ ]:
recipes.ingredients.str.len().describe()

,ingredients
count,173278.000000
mean,244.617926
std,146.705285
min,0.000000
25%,147.000000
50%,221.000000
75%,314.000000
max,9067.000000


The ingredient lists average 250 characters long, with a minimum of 0 and a maximum of nearly 10,000 characters!

Just out of curiosity, let's see which recipe has the longest ingredient list:

In [ ]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

We can do other aggregate explorations; for example, we can see how many of the recipes are for breakfast foods (using regular expression syntax to match both lowercase and capital letters):

In [ ]:
recipes.description.str.contains('[Bb]reakfast').sum()

3524

Or how many of the recipes list cinnamon as an ingredient:

In [ ]:
recipes.ingredients.str.contains('[Cc]innamon').sum()

10526

We could even look to see whether any recipes misspell the ingredient as "cinamon":

In [ ]:
recipes.ingredients.str.contains('[Cc]inamon').sum()

11

This is the type of data exploration that is possible with Pandas string tools.
It is data munging like this that Python really excels at.

### A Simple Recipe Recommender

Let's go a bit further, and start working on a simple recipe recommendation system: given a list of ingredients, we want to find any recipes that use all those ingredients.
While conceptually straightforward, the task is complicated by the heterogeneity of the data: there is no easy operation, for example, to extract a clean list of ingredients from each row.
So, we will cheat a bit: we'll start with a list of common ingredients, and simply search to see whether they are in each recipe's ingredient list.
For simplicity, let's just stick with herbs and spices for the time being:

In [ ]:
spice_list = ['salt', 'pepper', 'oregano', 'sage', 'parsley',
              'rosemary', 'tarragon', 'thyme', 'paprika', 'cumin']

We can then build a Boolean `DataFrame` consisting of `True` and `False` values, indicating whether each ingredient appears in the list:

In [ ]:
import re
spice_df = pd.DataFrame({
    spice: recipes.ingredients.str.contains(spice, re.IGNORECASE)
    for spice in spice_list})
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


Now, as an example, let's say we'd like to find a recipe that uses parsley, paprika, and tarragon.
We can compute this very quickly using the `query` method of ``DataFrame``s, discussed further in [High-Performance Pandas: `eval()` and `query()`](03.12-Performance-Eval-and-Query.ipynb):

In [ ]:
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

10

We find only 10 recipes with this combination. Let's use the index returned by this selection to discover the names of those recipes:

In [ ]:
recipes.name[selection.index]

,name
2069,"All cremat with a Little Gem, dandelion and wa..."
74964,Lobster with Thermidor butter
93768,Burton's Southern Fried Chicken with White Gravy
113926,Mijo's Slow Cooker Shredded Beef
137686,Asparagus Soup with Poached Eggs
140530,Fried Oyster Po’boys
158475,Lamb shank tagine with herb tabbouleh
158486,Southern fried chicken in buttermilk
163175,Fried Chicken Sliders with Pickles + Slaw
165243,Bar Tartine Cauliflower Salad


Now that we have narrowed down our recipe selection from 175,000 to 10, we are in a position to make a more informed decision about what we'd like to cook for dinner.

### Going Further with Recipes

Hopefully this example has given you a bit of a flavor (heh) of the types of data cleaning operations that are efficiently enabled by Pandas string methods.
Of course, building a robust recipe recommendation system would require a *lot* more work!
Extracting full ingredient lists from each recipe would be an important piece of the task; unfortunately, the wide variety of formats used makes this a relatively time-consuming process.
This points to the truism that in data science, cleaning and munging of real-world data often comprises the majority of the work—and Pandas provides the tools that can help you do this efficiently.